In [0]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [0]:
# Ler um arquivo em texto puro contendo o conteúdo a ser aprendido
file = open('got.txt').read()
file_len = len(file)
print('file_len =', file_len)

all_characters = ''.join(set(file))
n_characters = len(all_characters)

print(all_characters, n_characters)

file_len = 1694069
êoHWa‘sJ)ZSVeéjÃ–xDU!(t4ôóLçFB5-”;O—u“álkAÀEXâG’ÚrQNMPúwzq6õCíyÂv3bI1d2Ó :àpÁY.gKmRc,ÉTÍfnihã?
 95


In [0]:
import regex as re

word_dict = {}
word_list = []
ponctuation  = ["\n","\r",".",",","?","(", ")", "!", "-", "'", '"', "...", "–", ":", ";", "…", "“", "—",
               ":", "”", "’", "‘", "&", "%", "*", "@", "$", "[", "]", "`", "'", ">", "<", "{", "}", "#", "/", '\\', '°' ]
for p in ponctuation:
    file = file.replace(p, " " + p + " ")  

words = file.split(' ')
for word in words:
  word = re.sub(r'[^\w\s]','',word) 
  
  if word not in word_dict:
    word_list.append(word)
    word_dict[word] = len(word_list) - 1

for item in ponctuation:
  if item not in word_dict:
    word_list.append(item)
    word_dict[item] = len(word_list) - 1
    

In [0]:
chunk_len = 50

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    start_index = random.randint(0, len(word_list) - chunk_len)
    end_index = start_index + chunk_len + 1
    response = words[start_index : end_index]
    return response

print(random_chunk())
print(len(random_chunk()))

['enquanto', 'falava', ',', '', 'polindo', 'o', 'metal', 'até', 'soltar', 'um', '\n', 'brilho', 'escuro', '.', '', '', '–', '', 'Fiquei', 'contente', 'por', 'Bran', '.', '', 'Teria', 'ficado', 'orgulhosa', 'dele', '.', '', '\n', '', '–', '', 'Estou', 'sempre', 'orgulhosa', 'de', 'Bran', '', '–', '', 'Catelyn', 'respondeu', ',', '', 'observando', 'a', 'espada', 'enquanto', 'ele']
51


In [0]:

def word2tensor(word):
    one_hot = torch.zeros(len(word_list)).float()
    idx = word_dict[word]
    one_hot[idx] = 1
    return one_hot
  

In [0]:

def random_training_set():    
  
    chunk = random_chunk()
    inp = [word2tensor(word) for word in chunk[:-1]]
    target = [word2tensor(word) for word in chunk[1:]]
    
    return torch.stack(inp), torch.stack(target)
  
    

In [0]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers 
        self.lstm = nn.LSTM(input_size, hidden_size, n_layers)
        self.mlp = nn.Linear(hidden_size, output_size)
    
    def forward(self, X, H=None):
        out, hidden = self.lstm(X,H) 
        out = self.mlp(out[-1])
        return out, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [0]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  
    words = prime_str.split(' ')
    hidden = init_hidden()    
    prime_input = torch.stack([word2tensor(word) for word in words]).cuda()
    predicted = prime_str

    for w in prime_input:
        _, hidden = lstm(w.unsqueeze(0).unsqueeze(0), hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        inp = inp.unsqueeze(0).unsqueeze(0)
        output, hidden = lstm(inp, hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_word = word_list[torch.argmax(output)]
        predicted += ' ' + predicted_word
        inp = word2tensor(predicted_word).cuda()

    return predicted

In [0]:
def init_hidden():
  ho = Variable(torch.rand((n_layers,1,hidden_size)), requires_grad=False).cuda()
  hc = Variable(torch.rand((n_layers,1,hidden_size)), requires_grad=False).cuda()
  return (ho, hc)

In [0]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = init_hidden()
    lstm.zero_grad()
    loss = 0

    for i in range(len(inp)):
        output, hidden = lstm(inp[i].unsqueeze(0), hidden)
        loss += loss_metric(output, target[i].argmax().unsqueeze(0))
    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [0]:
n_epochs = 50000
print_every = 100
hidden_size = 512
n_layers = 1
lr = 0.0005

cuda = torch.device('cuda')
n_words = len(word_list)
lstm = LSTM(n_words, hidden_size, n_words, n_layers)
lstm.cuda()

optimizer = torch.optim.Adam(lstm.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss().cuda()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].unsqueeze(1).cuda(), example[1].unsqueeze(1).cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Os', 20), '\n')


 45s (100 0%) 6.7750]
Os . . . . . . . . . . . . . . . . . . . . 

 31s (200 0%) 6.8274]
Os 
 , , . . . . . . . . . . . . . . . . . 

 16s (300 0%) 6.2186]
Os 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 2s (400 0%) 6.4688]
Os 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 

 47s (500 1%) 5.5454]
Os . . 
 . . 
 . . 
 . . . 
 . . 
 . . . 
 

 33s (600 1%) 6.5242]
Os . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 

 18s (700 1%) 6.1348]
Os 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 
 . 

 4s (800 1%) 6.4226]
Os 
 . 
 
 . 
 . 
 
 . 
 . 
 
 . 
 . 
 
 . 

 49s (900 1%) 5.7328]
Os . 
 – – , , . 
 – – , , . 
 – – , , . 
 

 35s (1000 2%) 7.0654]
Os 
 – de . 
 – – de , . 
 – – de , . 
 – – de 

 20s (1100 2%) 6.3490]
Os , , , 
 – , , , 
 – , , , 
 – , , , 
 – 

 5s (1200 2%) 5.8279]
Os . 
 – de de . 
 – de de . 
 – de de . 
 – de de 

 51s (1300 2%) 5.7112]
Os . 
 – - se , o o . 
 – Não , , o o , e , e 

 36s (1400 2%) 5.8018]
Os 
 – , o 
 – disse . 
 – Não , de , e 
 – disse . 
 

 22s (1500 3%) 6.2001]
Os 
 – diss

In [0]:
# Com a rede treinada, gerar um texto longo

print(evaluate('O', 5500), '\n')

O irmão ergueu o vestido para que ela o 
 inspecionasse . 
 – Isto é uma beleza ! Toque - o . Vamos . Acaricie o tecido . 
 Dany o tocou . O tecido era tão macio que parecia correr - lhe pelos dedos como água . Não 
 conseguia se lembrar de alguma vez ter usado algo tão suave . Assustou - se . Afastou a mão . 
 – É mesmo meu ? 
 – Um presente do Magíster Illyrio – disse Viserys , sorrindo . Seu irmão estava de bom humor 
 naquela noite . – A cor realçará o violeta de seus olhos . E você também terá ouro e joias de todos 
 os tipos . Illyrio prometeu . Esta noite deve se parecer uma princesa . 
 Uma princesa , pensou Dany . Já se esquecera de como era aquilo . Talvez nunca tivesse 
 realmente sabido . 
 – Por que ele nos dá tanto ? – ela perguntou . – O que quer de nós ? – há quase meio ano que 
 viviam na casa do magíster , comiam de sua comida , eram paparicados por seus criados . Dany 
 tinha treze anos , idade suficiente para saber que tais presentes raramente vêm sem preço ali , na

##Anotações:
No inicio do treinamento a rede repete muito as palavras, principalmente os acentos, a rede parece aprender a chutar as palavras mais repetidas no texto. No meio do treino frases são geradas, mas o texto parece perder sentido no final das frases. Ja no final do treinamento a rede parecer aprender a decorar  pedaços do texto e sempre os repete.
Utilizando a LSTM com palavras a rede nao cometeu mais erros de sintaxe.

